In [43]:
import requests
from multiprocessing import Pool
from ConfigParser import ConfigParser
from urllib import quote_plus

In [54]:
def upcoming_forecast(args):
    """ Pulls upcoming forecast based on address. """
    config, address = args
    lat, lon = get_lat_long(config, address)
    resp = requests.get('http://api.openweathermap.org/data/2.5/forecast',
                        params={'lat': lat, 'lon': lon, 
                                'appid': config.get('openweather', 'api_key'),
                                'units': 'metric'})
    return (address, resp.json())

In [55]:
def get_config():
    """ Return my config object. """
    conf = ConfigParser()
    conf.read('config/prod.cfg')
    return conf

In [56]:
def get_lat_long(config, address):
    """ Returns lat and long from Google geocode API based on address. """
    qs_dict = {'address': quote_plus(address),
               'key': config.get('google', 'api_key'),
    }
    resp = requests.get('https://maps.googleapis.com/maps/api/geocode/json', params=qs_dict)
    try:
        lat, lon = resp.json().get('results')[0].get('geometry').get('location').values()
    except KeyError:
        raise Exception('Could not find address: %s', address)
    return lat, lon

In [59]:
def get_city_forecasts(addresses): 
    """ Returns forecasts when given a list of string addresses based on Google geocoding and openweather data. """
    config = get_config()
    pool = Pool(processes=4) 
    return pool.map(upcoming_forecast, [(config, addy) for addy in addresses])

In [61]:
my_weather = get_city_forecasts(['Los Angeles, CA', 'New York, NY', 'Berlin, Germany'])

In [62]:
type(my_weather)

list

In [63]:
for item in my_weather:
    print item[0]

Los Angeles, CA
New York, NY
Berlin, Germany
